In [1]:
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.optim as optim
import random
from transformers import BertTokenizer, BertModel
import json
import numpy as np
from tqdm import tqdm
import pickle
from sklearn.metrics import f1_score
import matplotlib.pyplot as plt
torch.manual_seed(1)

c:\Users\DELL\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = "cpu"

In [3]:
#check if cuda is available
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# print(device)

In [4]:
def argmax(vec):
    # return the argmax as a python int
    _, idx = torch.max(vec, 1)
    return idx.item()

def prepare_sequence(seq, to_ix):
    idxs = [to_ix[w] for w in seq]
    return torch.tensor(idxs, dtype=torch.long)

#### BiLSTM CRF model

In [5]:
class BiLSTM_CRF(nn.Module):

    def __init__(self, vocab_size, embedding_dim, hidden_dim, target_size, embedding_mat, start_tag, end_tag, tag_to_ix, batch_size=1, device='cpu'):
        super(BiLSTM_CRF, self).__init__()
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.vocab_size = vocab_size
        self.target_size = target_size
        self.batch_size = batch_size
        self.device = device
        self.tag_to_ix = tag_to_ix
        self.start_tag = start_tag
        self.end_tag = end_tag

        self.embedding = nn.Embedding.from_pretrained(embedding_mat, freeze=False).to(device)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim // 2,
                            num_layers=1, bidirectional=True).to(device)
        self.hidden2tag = nn.Linear(hidden_dim, target_size).to(device)

        self.transitions_to = nn.Parameter(torch.randn(target_size, target_size)).to(device)
        self.transitions_to.data[start_tag, :] = -10000
        self.transitions_to.data[:, end_tag] = -10000

        # self.transitions_from = nn.Parameter(torch.randn(target_size, target_size))
        # self.transitions_from.data[:, start_tag] = -10000
        # self.transitions_from.data[end_tag, :] = -10000

        self.hidden = self.init_hidden()
        
    def init_hidden(self):
        return (torch.randn(2, 1, self.hidden_dim // 2).to(self.device),
                torch.randn(2, 1, self.hidden_dim // 2).to(self.device))

    def get_lstm_features(self, sentence):
        self.hidden = self.init_hidden()
        embeds = self.embedding(sentence).view(len(sentence), 1, -1)
        #convert embeds to torch float32
        embeds = embeds.float()
        lstm_out, self.hidden = self.lstm(embeds, self.hidden)
        lstm_out = lstm_out.view(len(sentence), self.hidden_dim)
        lstm_feats = self.hidden2tag(lstm_out)
        return lstm_feats


    def _forward_algo(self, lstm_features):

        scores = torch.full((1, self.target_size), -10000.).to(self.device)
        scores[0][self.start_tag] = 0.

        forward_var = scores

        for feat in lstm_features:
            next_tag_var = self.transitions_to + feat.view(-1, 1).expand(-1, self.target_size) + forward_var.expand(self.target_size, -1)
            max_score = next_tag_var.max(dim=1).values.view(-1, 1)
            next_tag_var = next_tag_var - max_score
            forward_var = (max_score + torch.logsumexp(next_tag_var, dim=1).view(-1, 1)).view(1, -1)
            
        terminal_var = forward_var + (self.transitions_to[self.end_tag]).view(1, -1)
        alpha = terminal_var
        max_score = alpha.max()
        alpha = max_score + torch.logsumexp(alpha - max_score, dim=1)
        return alpha
    

    def _score_sentence(self, lstm_features, tags):
        score = torch.zeros(1).to(self.device)
        tags = torch.cat([torch.tensor([self.tag_to_ix['START_TAG']], dtype=torch.long).to(self.device), tags]).to(self.device)
        for i, feat in enumerate(lstm_features):
            score += self.transitions_to[tags[i + 1], tags[i]] + feat[tags[i + 1]]
                
        score += self.transitions_to[self.tag_to_ix['END_TAG'], tags[-1]]
        return score
    
    def neg_log_likelihood(self, sentence, tags):
        lstm_feats = self.get_lstm_features(sentence)
        forward_score = self._forward_algo(lstm_feats)
        gold_score = self._score_sentence(lstm_feats, tags)
        return forward_score - gold_score
    

    def _viterbi_decode(self, feats):
        backpointers = []

        # Initialize the viterbi variables in log space
        init_vvars = torch.full((1, self.target_size), -10000.).to(self.device)
        init_vvars[0][self.start_tag] = 0

        forward_var = init_vvars
        for feat in feats:
            bptrs_t = [] 
            viterbivars_t = [] 

            next_tag_var = self.transitions_to + forward_var.expand(self.target_size, -1)
            best_tag_id = torch.argmax(next_tag_var, dim=1)
            bptrs_t = best_tag_id
            viterbivars_t = next_tag_var[range(len(best_tag_id)), best_tag_id].view(1, -1)
            
            forward_var = (viterbivars_t + feat).view(1, -1)
            backpointers.append(bptrs_t)

        
        terminal_var = forward_var + self.transitions_to[self.end_tag]
        best_tag_id = argmax(terminal_var)
        path_score = terminal_var[0][best_tag_id]

       
        best_path = [best_tag_id]
        for bptrs_t in reversed(backpointers):
            best_tag_id = bptrs_t[best_tag_id]
            best_path.append(best_tag_id.item())
        
        start = best_path.pop()
        best_path.reverse()
        return path_score, best_path
        
    def forward(self, sentence):  # dont confuse this with _forward_alg above.
        # Get the emission scores from the BiLSTM
        lstm_feats = self.get_lstm_features(sentence)

        # Find the best path, given the features.
        score, tag_seq = self._viterbi_decode(lstm_feats)
        return score, tag_seq

        

In [6]:
train_data = json.load(open('../Dataset/BIO_Tagged/ATE_train.json', 'r'))
test_data = json.load(open('../Dataset/BIO_Tagged/ATE_test.json', 'r'))
val_data = json.load(open('../Dataset/BIO_Tagged/ATE_val.json', 'r'))

In [10]:
word_to_idx = pickle.load(open('../Utils/word_to_idx.pkl', 'rb'))

In [11]:
tag_to_ix = pickle.load(open('../Utils/tag_to_ix.pkl', 'rb'))

#### Embeddings

In [12]:
bert_embedding_mat = pickle.load(open('../Extracted Word Embeddings/bert_embedding_mat.pkl', 'rb'))
word2vec_embedding_mat = pickle.load(open('../Extracted Word Embeddings/word2vec_embedding_mat.pkl', 'rb'))
glove_embedding_mat = pickle.load(open('../Extracted Word Embeddings/glove_embedding_mat.pkl', 'rb'))

#### Glove

In [13]:
model = BiLSTM_CRF(len(word_to_idx), 300, 256, len(tag_to_ix), torch.tensor(glove_embedding_mat).to(device), tag_to_ix['START_TAG'], tag_to_ix['END_TAG'], tag_to_ix, device=device).to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, weight_decay=1e-4)

EPOCHS = 10
train_loss = []
val_loss = []
train_macro_f1 = []
val_macro_f1 = []

for epoch in tqdm(range(EPOCHS), desc='Epoch'):
    loss = 0
    f1 = 0
    for i, case in tqdm(enumerate(train_data), desc=f'Epoch {epoch + 1}/{EPOCHS}'):
        sentence = prepare_sequence(train_data[case]['text'].split(' '), word_to_idx)
        tags = prepare_sequence(train_data[case]['labels'], tag_to_ix)
        sentence = sentence.to(device)
        tags = tags.to(device)
        model.train(True)
        model.zero_grad()
        loss = model.neg_log_likelihood(sentence, tags)
        loss.backward()
        optimizer.step()
        loss += loss.item()
        with torch.no_grad():
            preds = model(sentence)[1]
            tags = tags.cpu().numpy()
            f1 += f1_score(tags, preds, average='macro')
    
    train_loss.append(loss/len(train_data))
    train_macro_f1.append(f1/len(train_data))
    
    model.eval()
    with torch.no_grad():
        loss = 0
        f1 = 0
        for case in val_data:
            sentence = prepare_sequence(val_data[case]['text'].split(' '), word_to_idx)
            tags = prepare_sequence(val_data[case]['labels'], tag_to_ix)
            setence = sentence.to(device)
            tags = tags.to(device)
            loss = model.neg_log_likelihood(sentence, tags)
            loss += loss.item()
            preds = model(sentence)[1]
            f1 += f1_score(tags.cpu().numpy(), preds, average='macro')
        val_macro_f1.append(f1/len(val_data))
        val_loss.append(loss/len(val_data))

    print()
    print(f'Train loss: {train_loss[-1]}, Val loss: {val_loss[-1]}')
    print(f'Train macro f1: {train_macro_f1[-1]}, Val macro f1: {val_macro_f1[-1]}')
    print(f'Epoch {epoch + 1}/{EPOCHS} done')

Epoch 1/10: 906it [00:24, 37.47it/s]?, ?it/s]
Epoch:  10%|█         | 1/10 [00:26<03:55, 26.18s/it]


Train loss: tensor([0.0435], grad_fn=<DivBackward0>), Val loss: tensor([0.0943])
Train macro f1: 0.5689398223325419, Val macro f1: 0.6274407505161015
Epoch 1/10 done


Epoch 2/10: 906it [00:21, 42.91it/s]
Epoch:  20%|██        | 2/10 [00:49<03:14, 24.31s/it]


Train loss: tensor([0.0239], grad_fn=<DivBackward0>), Val loss: tensor([0.0870])
Train macro f1: 0.7323482430685896, Val macro f1: 0.6746126532503784
Epoch 2/10 done


Epoch 3/10: 906it [00:21, 42.27it/s]
Epoch:  30%|███       | 3/10 [01:12<02:47, 23.90s/it]

...


Train loss: tensor([0.0068], grad_fn=<DivBackward0>), Val loss: tensor([0.0566])
Train macro f1: 0.9453833173860222, Val macro f1: 0.7046250986898516
Epoch 8/10 done


Epoch 9/10: 906it [00:25, 35.75it/s]
Epoch:  90%|█████████ | 9/10 [04:32<00:29, 29.67s/it]


Train loss: tensor([0.0055], grad_fn=<DivBackward0>), Val loss: tensor([0.0544])
Train macro f1: 0.9640079294147783, Val macro f1: 0.7236335294969286
Epoch 9/10 done


Epoch 10/10: 906it [00:23, 38.50it/s]
Epoch: 100%|██████████| 10/10 [04:58<00:00, 29.87s/it]


Train loss: tensor([0.0035], grad_fn=<DivBackward0>), Val loss: tensor([0.0526])
Train macro f1: 0.9730837339054916, Val macro f1: 0.7275297537996208
Epoch 10/10 done


In [14]:
torch.save(model, 'Non Trainable Embeddings/Glove/bilstm_crf_glove.pt')
pickle.dump(train_loss, open('Non Trainable Embeddings/Glove/train_loss.pkl', 'wb'))
pickle.dump(val_loss, open('Non Trainable Embeddings/Glove/val_loss.pkl', 'wb'))
pickle.dump(train_macro_f1, open('Non Trainable Embeddings/Glove/train_f1.pkl', 'wb'))
pickle.dump(val_macro_f1, open('Non Trainable Embeddings/Glove/val_f1.pkl', 'wb'))

#### Word2vec

In [15]:
model = BiLSTM_CRF(len(word_to_idx), 300, 256, len(tag_to_ix), torch.tensor(word2vec_embedding_mat).to(device), tag_to_ix['START_TAG'], tag_to_ix['END_TAG'], tag_to_ix, device=device).to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, weight_decay=1e-4)

EPOCHS = 10
train_loss = []
val_loss = []
train_macro_f1 = []
val_macro_f1 = []

for epoch in tqdm(range(EPOCHS), desc='Epoch'):
    loss = 0
    f1 = 0
    for i, case in tqdm(enumerate(train_data), desc=f'Epoch {epoch + 1}/{EPOCHS}'):
        sentence = prepare_sequence(train_data[case]['text'].split(' '), word_to_idx)
        tags = prepare_sequence(train_data[case]['labels'], tag_to_ix)
        sentence = sentence.to(device)
        tags = tags.to(device)
        model.train(True)
        model.zero_grad()
        loss = model.neg_log_likelihood(sentence, tags)
        loss.backward()
        optimizer.step()
        loss += loss.item()
        with torch.no_grad():
            preds = model(sentence)[1]
            tags = tags.cpu().numpy()
            f1 += f1_score(tags, preds, average='macro')
    
    train_loss.append(loss/len(train_data))
    train_macro_f1.append(f1/len(train_data))
    
    model.eval()
    with torch.no_grad():
        loss = 0
        f1 = 0
        for case in val_data:
            sentence = prepare_sequence(val_data[case]['text'].split(' '), word_to_idx)
            tags = prepare_sequence(val_data[case]['labels'], tag_to_ix)
            setence = sentence.to(device)
            tags = tags.to(device)
            loss = model.neg_log_likelihood(sentence, tags)
            loss += loss.item()
            preds = model(sentence)[1]
            f1 += f1_score(tags.cpu().numpy(), preds, average='macro')
        val_macro_f1.append(f1/len(val_data))
        val_loss.append(loss/len(val_data))

    print()
    print(f'Train loss: {train_loss[-1]}, Val loss: {val_loss[-1]}')
    print(f'Train macro f1: {train_macro_f1[-1]}, Val macro f1: {val_macro_f1[-1]}')
    print(f'Epoch {epoch + 1}/{EPOCHS} done')

Epoch 1/10: 906it [00:26, 34.67it/s]?, ?it/s]
Epoch:  10%|█         | 1/10 [00:28<04:16, 28.53s/it]


Train loss: tensor([0.0153], grad_fn=<DivBackward0>), Val loss: tensor([0.0623])
Train macro f1: 0.5148400644199287, Val macro f1: 0.6226556215851898
Epoch 1/10 done


Epoch 2/10: 906it [00:28, 32.33it/s]
Epoch:  20%|██        | 2/10 [00:59<03:57, 29.74s/it]


Train loss: tensor([0.0139], grad_fn=<DivBackward0>), Val loss: tensor([0.0658])
Train macro f1: 0.7332052658651897, Val macro f1: 0.6673895721582574
Epoch 2/10 done


Epoch 3/10: 906it [00:29, 30.70it/s]
Epoch:  30%|███       | 3/10 [01:31<03:36, 30.96s/it]

...


Train loss: tensor([0.0040], grad_fn=<DivBackward0>), Val loss: tensor([0.0456])
Train macro f1: 0.9471715801605495, Val macro f1: 0.7170925207976782
Epoch 8/10 done


Epoch 9/10: 906it [00:34, 26.45it/s]
Epoch:  90%|█████████ | 9/10 [05:00<00:35, 35.20s/it]


Train loss: tensor([0.0038], grad_fn=<DivBackward0>), Val loss: tensor([0.0407])
Train macro f1: 0.9638492751434581, Val macro f1: 0.720509042653718
Epoch 9/10 done


Epoch 10/10: 906it [00:40, 22.41it/s]
Epoch: 100%|██████████| 10/10 [05:45<00:00, 34.52s/it]


Train loss: tensor([0.0042], grad_fn=<DivBackward0>), Val loss: tensor([0.0289])
Train macro f1: 0.9703085228004333, Val macro f1: 0.7296129764487568
Epoch 10/10 done


In [16]:
torch.save(model, 'Non Trainable Embeddings/Word2vec/bilstm_crf_word2vec.pt')
pickle.dump(train_loss, open('Non Trainable Embeddings/Word2vec/train_loss.pkl', 'wb'))
pickle.dump(val_loss, open('Non Trainable Embeddings/Word2vec/val_loss.pkl', 'wb'))
pickle.dump(train_macro_f1, open('Non Trainable Embeddings/Word2vec/train_f1.pkl', 'wb'))
pickle.dump(val_macro_f1, open('Non Trainable Embeddings/Word2vec/val_f1.pkl', 'wb'))

#### Bert

In [17]:
model = BiLSTM_CRF(len(word_to_idx), 768, 512, len(tag_to_ix), torch.tensor(bert_embedding_mat).to(device), tag_to_ix['START_TAG'], tag_to_ix['END_TAG'], tag_to_ix, device=device).to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, weight_decay=1e-4)

EPOCHS = 10
train_loss = []
val_loss = []
train_macro_f1 = []
val_macro_f1 = []

for epoch in tqdm(range(EPOCHS), desc='Epoch'):
    loss = 0
    f1 = 0
    for i, case in tqdm(enumerate(train_data), desc=f'Epoch {epoch + 1}/{EPOCHS}'):
        sentence = prepare_sequence(train_data[case]['text'].split(' '), word_to_idx)
        tags = prepare_sequence(train_data[case]['labels'], tag_to_ix)
        sentence = sentence.to(device)
        tags = tags.to(device)
        model.train(True)
        model.zero_grad()
        loss = model.neg_log_likelihood(sentence, tags)
        loss.backward()
        optimizer.step()
        loss += loss.item()
        with torch.no_grad():
            preds = model(sentence)[1]
            tags = tags.cpu().numpy()
            f1 += f1_score(tags, preds, average='macro')
    
    train_loss.append(loss/len(train_data))
    train_macro_f1.append(f1/len(train_data))
    
    model.eval()
    with torch.no_grad():
        loss = 0
        f1 = 0
        for case in val_data:
            sentence = prepare_sequence(val_data[case]['text'].split(' '), word_to_idx)
            tags = prepare_sequence(val_data[case]['labels'], tag_to_ix)
            setence = sentence.to(device)
            tags = tags.to(device)
            loss = model.neg_log_likelihood(sentence, tags)
            loss += loss.item()
            preds = model(sentence)[1]
            f1 += f1_score(tags.cpu().numpy(), preds, average='macro')
        val_macro_f1.append(f1/len(val_data))
        val_loss.append(loss/len(val_data))

    print()
    print(f'Train loss: {train_loss[-1]}, Val loss: {val_loss[-1]}')
    print(f'Train macro f1: {train_macro_f1[-1]}, Val macro f1: {val_macro_f1[-1]}')
    print(f'Epoch {epoch + 1}/{EPOCHS} done')

Epoch 1/10: 906it [00:45, 19.78it/s]?, ?it/s]
Epoch:  10%|█         | 1/10 [00:49<07:23, 49.30s/it]


Train loss: tensor([0.0358], grad_fn=<DivBackward0>), Val loss: tensor([0.0815])
Train macro f1: 0.36694032087374634, Val macro f1: 0.3531173110138638
Epoch 1/10 done


Epoch 2/10: 906it [00:53, 16.94it/s]
Epoch:  20%|██        | 2/10 [01:46<07:12, 54.02s/it]


Train loss: tensor([0.0347], grad_fn=<DivBackward0>), Val loss: tensor([0.0819])
Train macro f1: 0.40508630161526976, Val macro f1: 0.37404826273550174
Epoch 2/10 done


Epoch 3/10: 906it [00:54, 16.64it/s]
Epoch:  30%|███       | 3/10 [02:44<06:31, 55.93s/it]

...


Train loss: tensor([0.0323], grad_fn=<DivBackward0>), Val loss: tensor([0.1285])
Train macro f1: 0.7718432649084743, Val macro f1: 0.5090840071235446
Epoch 8/10 done


Epoch 9/10: 906it [00:44, 20.55it/s]
Epoch:  90%|█████████ | 9/10 [08:22<00:54, 54.53s/it]


Train loss: tensor([0.0288], grad_fn=<DivBackward0>), Val loss: tensor([0.1391])
Train macro f1: 0.8018852724784512, Val macro f1: 0.49414300920914944
Epoch 9/10 done


Epoch 10/10: 906it [00:47, 18.91it/s]
Epoch: 100%|██████████| 10/10 [09:14<00:00, 55.43s/it]


Train loss: tensor([0.0343], grad_fn=<DivBackward0>), Val loss: tensor([0.0996])
Train macro f1: 0.8237810294876755, Val macro f1: 0.5400539278465898
Epoch 10/10 done


In [18]:
torch.save(model, 'Non Trainable Embeddings/Bert/bilstm_crf_bert.pt')
pickle.dump(train_loss, open('Non Trainable Embeddings/Bert/train_loss.pkl', 'wb'))
pickle.dump(val_loss, open('Non Trainable Embeddings/Bert/val_loss.pkl', 'wb'))
pickle.dump(train_macro_f1, open('Non Trainable Embeddings/Bert/train_f1.pkl', 'wb'))
pickle.dump(val_macro_f1, open('Non Trainable Embeddings/Bert/val_f1.pkl', 'wb'))